In [1]:
import cv2
from batch_face import (
    RetinaFace,
    drawLandmark_multiple,
    LandmarkPredictor,
    SixDRep
)
import os

In [5]:
def draw_landmarks(faces, landmarks, pose, img):
    head_pose = SixDRep(gpu_id=-1) # change from 0 to gpu_id=-1 for Mac users
    for face, landmark in zip(faces, landmarks):
        img = drawLandmark_multiple(img, face[0], landmark)
        head_pose.plot_pose_cube(img, face[0], pose['pitch'], pose['yaw'], pose['roll'])
    return img

In [8]:
if __name__ == "__main__":
    output_dir = "./batch_img_results"
    os.makedirs(output_dir, exist_ok=True)

    predictor = LandmarkPredictor(gpu_id=-1) # change from 0 to gpu_id=-1 for Mac users
    detector = RetinaFace(gpu_id=-1) # change from 0 to gpu_id=-1 for Mac users
    head_pose = SixDRep(gpu_id=-1) # change from 0 to gpu_id=-1 for Mac users

    all_images = []
    names = os.listdir("batch_img_intake")

    for name in names:
        img = cv2.imread(os.path.join("batch_img_intake", name))
        all_images.append(img)

    all_faces = detector.pseudo_batch_detect(
        all_images, cv=True, threshold=0.9
    )  # batch input
    all_results = predictor(all_faces, all_images, from_fd=True)
    poses = head_pose(all_faces, all_images)

    assert len(all_results) == len(all_faces)

    for faces, landmarks, img, name, pose in zip(all_faces, all_results, all_images, names, poses):
        assert len(faces) == len(landmarks)
        img = draw_landmarks(faces,landmarks,pose[0],img)
        cv2.imwrite(os.path.join(output_dir, name), img)
        pitch, yaw, roll = (pose[0]['pitch'], pose[0]['yaw'], pose[0]['roll'])
        print(f"File Name: {name}, Pitch: {pitch}, Yaw: {yaw}, Roll {roll}")
    

File Name: obama.png, Pitch: -10.937263488769531, Yaw: 5.830609321594238, Roll 23.617137908935547
File Name: obama.jpg, Pitch: -13.801589965820312, Yaw: 0.683098554611206, Roll -1.1091930866241455
File Name: trump.jpg, Pitch: -12.918827056884766, Yaw: -8.755852699279785, Roll 1.0519715547561646
